# FANDANGO


# Group 1:
Zachary Wilkerson, Tara Bode, Hankun Li

### Some things we want to know:
- Of the Movies Now Playing, how highly rated were they by critics and audience members?
- Of the reviews from audience members, how many occurrences of keywords are there of each review and to what extent (i.e. extra positive vs. positive)?

In [1]:
import pandas as pd
import time # needed for time.sleep
from selenium import webdriver
# new driver (opens browser window)
driver = webdriver.Chrome(r'/Users/Tara/Desktop/Summer A 2021/ACG7849- Web Crawling/selenium_files/chromedriver')
driver.get("https://www.fandango.com/movie-reviews")

In [2]:
with open(r'/Users/Tara/Desktop/Summer A 2021/ACG7849- Web Crawling/selenium_files/jquery.js', 'r') as jquery_js:
    driver.execute_script( jquery_js.read() )

In [ ]:
# For jquery debugger
'''
var script = document.createElement("script")
script.type = "text/javascript";
script.src = "https://code.jquery.com/jquery-3.6.0.js";
document.getElementsByTagName("head")[0].appendChild(script);
'''

In [ ]:
# figure out the jquery to get all links from the main page
# do this in the debugger
#javascript (browser)
$("tbody.mrp__table--mobile-block a")

In [ ]:
#javascript
#javascript (browser)
els =$("tbody.mrp__table--mobile-block a").filter(function(index){return (index % 4 == 0);})

all_links = []
els.each( function(i,el) { all_links.push( el.href )  }   )

In [7]:
# write a function to get all links into python/selenium
# to get the href of an element we don't need jquery, just el.href (not $(el).href )
myJs = '''
els =$("tbody.mrp__table--mobile-block a").filter(function(index){return (index % 4 == 0);})

all_links = []
els.each( function(i,el) { all_links.push( el.href )  }   )
return all_links;
'''
links = driver.execute_script( myJs )

In [8]:
# Correct version

myJs_details ='''
cells = els = $("section.movie-reviews")

details = {}

cells.each( 
    function(index, el) {  
        criticReviews = $( el ).find("span.movie-ratings-tabs__rating").text().slice(0,3)
        audienceReviews = $( el ).find("span.movie-ratings-tabs__rating").text().slice(3,6)
        if (criticReviews != "") { 
            details['Critics'] = criticReviews.trim();
            details['Audience'] = audienceReviews.trim();
        }               
    }      
)
details['Movie Title'] = $('a[class="js-hearts-heart favorite-add icon icon-fave-heart--white-outline"]').attr('data-name')
return details;

'''

reviews = '''
elem = $("p.rt-audience-reviews__content")
review = []
review.push($('a[class="js-hearts-heart favorite-add icon icon-fave-heart--white-outline"]').attr('data-name'))
elem.each (
     function(index, el) { 
         review.push(el.innerText)
     })
return review
'''


[None]

In [5]:
# create keyword lists
positive = ['love','fun','good','quality','great']
extra_positive = ['phenomenal','top','heartwarming','favorite','awesome']

negative = [ 'horrible', 'predictable', 'boring', 'bad']
extra_negative = [ 'disappointing', 'worst', 'waste', 'awful','terrible']

In [9]:
review_list = []
movie_list = []
movieData = pd.DataFrame(columns = ['Movie Title','Critics', 'Audience', 'Positive Reviews','Extra Positive Reviews','Negative Reviews','Extra Negative Reviews'] )

for link in links[0:2]:
    # go to url
    driver.get( link  )
    # some time to finish loading
    driver.implicitly_wait(1)
    # normally inject jquery, but it is already there
    # skip
    # get details
    d = driver.execute_script( myJs_details )  
    
    positiveReviewsCnt = 0
    extraPositiveReviewsCnt = 0
    negativeReviewsCnt = 0
    extraNegativeReviewsCnt = 0

    test = {}
    
    f=driver.execute_script( reviews )    
    movie_list.append(f[0])
    review_list.append(f[1:])
    
    for review in review_list:
        for word in positive: 
            positiveReviewsCnt += review.count(word)
        for word in extra_positive: 
            extraPositiveReviewsCnt += review.count(word)
        for word in negative: 
            negativeReviewsCnt += review.count(word) 
        for word in extra_negative: 
            extraNegativeReviewsCnt += review.count(word)

    test['Movie Title'] = f[0]
    test['Positive Reviews'] = positiveReviewsCnt   
    test['Extra Positive Reviews'] = extraPositiveReviewsCnt
    test['Negative Reviews'] = negativeReviewsCnt
    test['Extra Negative Reviews'] =extraNegativeReviewsCnt
    
    d.update(test) 
    movieData = movieData.append (d, ignore_index = True  )    

In [10]:
movieData

,Movie Title,Critics,Audience,Positive Reviews,Extra Positive Reviews,Negative Reviews,Extra Negative Reviews
0,Spiral: SAW (2021),37%,76%,0,0,0,0
1,Wrath of Man (2021),66%,91%,0,0,0,0


In [ ]:
d

In [ ]:
test